In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#image augmentation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest')


img = load_img('/content/drive/MyDrive/Artificial Intelligence/Deep Learning/Keras/Cat Vs Dog/File/train/Cat/cat.201.jpg')

x = img_to_array(img)
x = x.reshape((1,) + x.shape)

i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir = '/content/drive/MyDrive/Artificial Intelligence/Deep Learning/Keras/Cat Vs Dog/File/Extra_2', save_prefix = 'cat', save_format = 'jpeg'):
    i += 1
    if i>20:
        break

#Training a small convnet from scratch: 80% accuracy in 40 lines of code

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras import optimizers

img_width, img_height = 150, 150

train_data_dir = '/content/drive/MyDrive/Artificial Intelligence/Deep Learning/Keras/Cat Vs Dog/File/train'
validation_data_dir = '/content/drive/MyDrive/Artificial Intelligence/Deep Learning/Keras/Cat Vs Dog/File/Validation'

nb_train_samples = 1000
nb_validation_samples = 800
epochs = 20
batch_size = 8

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)
model = Sequential()
model.add(Conv2D(32,(3,3), input_shape = input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy'])

In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'binary')

model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs  = epochs,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size
)

Found 1000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
125/125 [==============================] - 383s 3s/step - loss: 0.6984 - accuracy: 0.5450 - val_loss: 0.6755 - val_accuracy: 0.5288
Epoch 2/20
125/125 [==============================] - 39s 315ms/step - loss: 0.6800 - accuracy: 0.5760 - val_loss: 0.6485 - val_accuracy: 0.5975
Epoch 3/20
125/125 [==============================] - 40s 318ms/step - loss: 0.6562 - accuracy: 0.6420 - val_loss: 0.6190 - val_accuracy: 0.6400
Epoch 4/20
125/125 [==============================] - 40s 317ms/step - loss: 0.6463 - accuracy: 0.6260 - val_loss: 0.6215 - val_accuracy: 0.6388
Epoch 5/20
125/125 [==============================] - 40s 317ms/step - loss: 0.6400 - accuracy: 0.6600 - val_loss: 0.5957 - val_accuracy: 0.6600
Epoch 6/20
125/125 [==============================] - 40s 316ms/step - loss: 0.6186 - accuracy: 0.6550 - val_loss: 0.6278 - val_accuracy: 0.6587
Epoch 7/20
125/125 [==============================] - 39s 315ms/step - loss: 0.6207 - accuracy: 0.6760 - val_loss: 0.6515 - val_accu

#Using the bottleneck features of a pre-trained network: 90% accuracy in a minute

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications
import numpy as np

img_width, img_height = 150, 150

top_model_weights_path = '/content/drive/MyDrive/Artificial Intelligence/Deep Learning/Keras/Cat Vs Dog/File/Save_File/bottleneck_fc_model.h5'
train_data_dir = '/content/drive/MyDrive/Artificial Intelligence/Deep Learning/Keras/Cat Vs Dog/File/train'
validation_data_dir = '/content/drive/MyDrive/Artificial Intelligence/Deep Learning/Keras/Cat Vs Dog/File/Validation'
nb_train_samples = 1000
nb_validation_samples = 800
epochs = 25
batch_size = 10
def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale = 1./255)
    model = applications.VGG16(include_top = False, weights = 'imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size = (img_width, img_height),
        batch_size = batch_size,
        class_mode = None,
        shuffle = False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples//batch_size)
    np.save(open('/content/drive/MyDrive/Artificial Intelligence/Deep Learning/Keras/Cat Vs Dog/File/Save_File/bottleneck_features_train.npy','wb'),
            bottleneck_features_train)
    
    generator  = datagen.flow_from_directory(
        validation_data_dir,
        target_size = (img_width, img_height),
        batch_size = batch_size,
        class_mode = None,
        shuffle = False)
    
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples //batch_size)
    np.save(open('/content/drive/MyDrive/Artificial Intelligence/Deep Learning/Keras/Cat Vs Dog/File/Save_File/bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)
    
def train_top_model():
    train_data = np.load(open('/content/drive/MyDrive/Artificial Intelligence/Deep Learning/Keras/Cat Vs Dog/File/Save_File/bottleneck_features_train.npy','rb'))
    train_labels = np.array(
        [0] * (nb_train_samples //2) + [1] * (nb_train_samples // 2))
    
    validation_data = np.load(open('/content/drive/MyDrive/Artificial Intelligence/Deep Learning/Keras/Cat Vs Dog/File/Save_File/bottleneck_features_validation.npy','rb'))
    validation_labels = np.array(
        [0] * (nb_validation_samples//2) + [1] * (nb_validation_samples //2))
    
    model = Sequential()
    model.add(Flatten(input_shape = train_data.shape[1:]))
    model.add(Dense(256, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation = 'sigmoid'))

    model.compile(optimizer = 'rmsprop',
                  loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    model.fit(train_data, train_labels,
              epochs  = epochs,
              batch_size = batch_size,
              validation_data = (validation_data, validation_labels))
    
    model.save_weights(top_model_weights_path)
    model.evaluate(validation_data, validation_labels)

save_bottlebeck_features()
train_top_model()

Found 1000 images belonging to 2 classes.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


Found 800 images belonging to 2 classes.
Epoch 1/25
100/100 [==============================] - 3s 21ms/step - loss: 1.6401 - accuracy: 0.6482 - val_loss: 0.6825 - val_accuracy: 0.6988
Epoch 2/25
100/100 [==============================] - 2s 20ms/step - loss: 0.4506 - accuracy: 0.8103 - val_loss: 0.3139 - val_accuracy: 0.8500
Epoch 3/25
100/100 [==============================] - 2s 21ms/step - loss: 0.3421 - accuracy: 0.8480 - val_loss: 0.3100 - val_accuracy: 0.8788
Epoch 4/25
100/100 [==============================] - 2s 20ms/step - loss: 0.2731 - accuracy: 0.8960 - val_loss: 0.3021 - val_accuracy: 0.8913
Epoch 5/25
100/100 [==============================] - 2s 20ms/step - loss: 0.2556 - accuracy: 0.9175 - val_loss: 0.3344 - val_accuracy: 0.8700
Epoch 6/25
100/100 [==============================] - 2s 20ms/step - loss: 0.1924 - accuracy: 0.9191 - val_loss: 0.3901 - val_accuracy: 0.8850
Epoch 7/25
100/100 [==============================] - 2s 21ms/step - loss: 0.1635 - accuracy: 0.9424 

In [ ]:
train_data = np.load(open('/content/drive/MyDrive/Artificial Intelligence/Deep Learning/Keras/Cat Vs Dog/File/Save_File/bottleneck_features_train.npy','rb'))

In [ ]:
train_data.shape[1:]

(4, 4, 512)

In [ ]:
nb_train_samples = 1000

train_labels = np.array(
        [0] * (nb_train_samples //2) + [1] * (nb_train_samples // 2))

In [ ]:
train_labels

In [ ]:
[2] * (nb_train_samples //2)

#Fine-tuning the top layers of a a pre-trained network

In [ ]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.models import Model

top_model_weights_path =  '/content/drive/MyDrive/Artificial Intelligence/Deep Learning/Keras/Cat Vs Dog/File/Save_File/bottleneck_fc_model.h5'
img_width, img_height = 150,150

train_data_dir = '/content/drive/MyDrive/Artificial Intelligence/Deep Learning/Keras/Cat Vs Dog/File/train'
validation_data_dir = '/content/drive/MyDrive/Artificial Intelligence/Deep Learning/Keras/Cat Vs Dog/File/Validation'
nb_train_samples = 1000
nb_validation_samples = 800
epochs = 50
batch_size = 16

model = applications.VGG16(weights = 'imagenet', include_top = False, input_shape = (150,150,3))
print('Model Loaded.')

top_model = Sequential()
top_model.add(Flatten(input_shape = model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation = 'sigmoid'))

top_model.load_weights(top_model_weights_path)

model = Model(inputs = model.input, outputs = top_model(model.output))

for layer in model.layers[:25]:
    layer.trainable  = False

model.compile(loss='binary_crossentropy',
              optimizer = optimizers.SGD(lr=1e-4, momentum = 0.9),
              metrics = ['accuracy'])

train_datagen = ImageDataGenerator(
    rescale = 1. / 255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./ 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size  = (img_height, img_width),
    batch_size = batch_size,
    class_mode = 'binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size,
    class_mode = 'binary')

model.fit_generator(
    train_generator,
    epochs = epochs,
    validation_data = validation_generator)

Model Loaded.
Found 1000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
63/63 [==============================] - 895s 14s/step - loss: 0.2806 - accuracy: 0.9396 - val_loss: 0.9377 - val_accuracy: 0.8800
Epoch 2/50
63/63 [==============================] - 389s 6s/step - loss: 0.3837 - accuracy: 0.9305 - val_loss: 0.9377 - val_accuracy: 0.8800
Epoch 3/50
63/63 [==============================] - 374s 6s/step - loss: 0.4125 - accuracy: 0.9264 - val_loss: 0.9377 - val_accuracy: 0.8800
Epoch 4/50
63/63 [==============================] - 373s 6s/step - loss: 0.2912 - accuracy: 0.9352 - val_loss: 0.9377 - val_accuracy: 0.8800
Epoch 5/50
63/63 [==============================] - 375s 6s/step - loss: 0.3283 - accuracy: 0.9390 - val_loss: 0.9377 - val_accuracy: 0.8800
Epoch 6/50
63/63 [==============================] - 375s 6s/step - loss: 0.5434 - accuracy: 0.9167 - val_loss: 0.9377 - val_accuracy: 0.8800
Epoch 7/50
63/63 [==============================] - 375s 6s/step - loss: 0.5533 - accuracy: 0.9075 - val_loss: 0.9377 - val_accuracy: 0.8800
Epoch 8/50
6